In [1]:
cd ../Localization_and_Detection/NoahSim/

/Users/noahkasmanoff/BurstCube/Simulation/Localization_and_Detection/NoahSim


In [2]:
import GRBgenerator

Below is the fastcube class, simulations for burstcube that run really fast but still pretty accurate. What sorts of customizations should I include?

Sample, samples n, 

Keep that test feature: 
maybe have a bruteforce vs. quicker version, all about fullsky sims now though. 

Also need to remember how I treated past horizon spots, and what the goal should be for those. 

In [5]:
#The following cell contains the "FastCube" class. This is the simulation I hope to use to be able to run quicker simulations. 

import numpy as np
import healpy as hp
import burstutils as bf
import random as rand
import statistics as s
import time as time

class FastCube():

    def __init__(self,background,dettilt,alternating=False):
        if alternating == False:
            self.tilt = np.deg2rad(dettilt)
            self.tiltA = self.tiltB = self.tiltC = self.tiltD = self.tilt
        
        else:
            self.tiltB = (float(input("Please enter the second tilt (deg) ")))
            self.tiltB = np.deg2rad(self.tiltB)
            self.tiltC = self.tiltA = np.deg2rad(dettilt)
            self.tiltD = self.tiltB
        
        self.zenith = [0 , 0]
        self.bg = background


        
    @property
    def detA(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltA , self.zenith[1] ]
    @property 
    def detB(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltB , self.zenith[1] + np.pi/2 ]
    @property
    def detC(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltC , self.zenith[1] + np.pi ]
    @property 
    def detD(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltD , self.zenith[1] + 3*np.pi/2 ]
    @property
    def normA(self):
        return  hp.ang2vec(self.detA[0],self.detA[1])
    @property 
    def normB(self):
        return  hp.ang2vec(self.detB[0],self.detB[1])
    @property
    def normC(self):
        return  hp.ang2vec(self.detC[0],self.detC[1])
    @property 
    def normD(self):
        return  hp.ang2vec(self.detD[0],self.detD[1])

    
    @property
    def dets(self):
        return [self.normA,self.normB,self.normC,self.normD] 
    
    
    
    def response2GRB(self, GRB, test=True):   #is this how I inherit? 
        start = time.time()
        #first need to include the GRB.
       
        """
        Using least squares regression, respond2GRB will determine the sky position of an array of GRB sources assuming some inherent background noise within 
        detectors, along with fluctuations of either Gaussian or Poissonian nature. 

        Parameters
        ----------
        GRB : object
            An instance of the separately defined "GRBs" class that contains a number of evenly spaced sky positions of a given strength. 
        
        test : boolean 
            For sanity purposes, if the simulation seems to give unrealistic results, switching to test mode allows for much quicker sampling, allowing it easier to spot potential errors. 
        
        

        Returns
        ----------
        localizationerrors : array
            numpy array that contains the average localization uncertainty at each sky position. 
        
        Additionally, response2GRB will print the sky position it is currently sampling, along with the average offset of localizations at that spot. 
        
        """
        
        if test:
            sample = 1
            samples = 50  #times  per sky pos
            bottheta = 0
            toptheta = 90
            botphi = 0 
            topphi = 360
            botA = 0
            topA = 1000
            ntheta = 10   #over sky chi points
            nphi = 37
            nA = 100

        else:
            sample = len(GRB.sourceangs) 
            samples = 30 #times  per sky pos
            bottheta = 0
            toptheta = 90
            botphi = 0 
            topphi = 360
            botA = 400
            topA = 1000
            ntheta = 31   #over sky chi points
            nphi = 120
            nA = 12
        self.localizationerrors = []    
        self.X_all = []
        self.y_all = []
        for i in range(sample):
            sourceAng = GRB.sourceangs[i]
          #  print("Testing " + str(np.rad2deg(sourceAng)))
           #this check passes.       

            
           # print("Testing at " + str(np.rad2deg(GRB.sourceangs)))
            sourcexyz = hp.ang2vec(sourceAng[0],sourceAng[1]) #cartesian position of the burst
            loop = 0 #I'm going to want to sample each sky position more than once,
                    #here's where I define how many times that is
            locunc = []
            while loop<samples:
                sepA=bf.angle(sourcexyz,self.normA)
                   # print("separation from A is " + str(np.rad2deg(sepA)))
                   #this check passes.  
               
                if sepA < np.pi/2: # meaning if >90, would not be facing detector.
                    dtheoryA=GRB.Ao*bf.response(bf.angle(sourcexyz,self.normA))  #still need to define strength, brb and gonna do that 
                else: #like I was saying, has to face it!
                    dtheoryA = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                countsA = dtheoryA + self.bg #another artifact, incl this background effect somewhere
                unccountsA = np.sqrt(countsA)
                detactualA = rand.gauss(countsA,unccountsA)  #there is a lot of noise, present, updating it now. 
                if detactualA-self.bg < 0:
                    detactualA = self.bg
                    
                detcountsA = detactualA
                
                sepB=bf.angle(sourcexyz,self.normB)
                   # print("separation from B is " + str(np.rad2deg(sepB)))
                   #this check passes.  
               
                if sepB < np.pi/2: # meaning if >90, would not be facing detector.
                    dtheoryB=GRB.Ao*bf.response(bf.angle(sourcexyz,self.normB))  #still need to define strength, brb and gonna do that 
                else: #like I was saying, has to face it!
                    dtheoryB = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                countsB = dtheoryB + self.bg #another artifact, incl this background effect somewhere
                unccountsB = np.sqrt(countsB)
                detactualB = rand.gauss(countsB,unccountsB)  #there is a lot of noise, present, updating it now. 
                if detactualB-self.bg < 0:
                    detactualB = self.bg
                    
                detcountsB = detactualB

                sepC=bf.angle(sourcexyz,self.normC)
                   # print("separation from C is " + str(np.rad2deg(sepC)))
                   #this check passes.  
               
                if sepC < np.pi/2: # meaning if >90, would not be facing detector.
                    dtheoryC=GRB.Ao*bf.response(bf.angle(sourcexyz,self.normC))  #still need to define strength, brb and gonna do that 
                else: #like I was saying, has to face it!
                    dtheoryC = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                countsC = dtheoryC + self.bg #another artifact, incl this background effect somewhere
                unccountsC = np.sqrt(countsC)
                detactualC = rand.gauss(countsC,unccountsC)  #there is a lot of noise, present, updating it now. 
                if detactualC-self.bg < 0:
                    detactualC = self.bg
                    
                detcountsC = detactualC
                
                sepD=bf.angle(sourcexyz,self.normD)
                   # print("separation from D is " + str(np.rad2deg(sepD)))
                   #this check passes.  
               
                if sepD < np.pi/2: # meaning if >90, would not be facing detector.
                    dtheoryD=GRB.Ao*bf.response(bf.angle(sourcexyz,self.normD))  #still need to define strength, brb and gonna do that 
                else: #like I was saying, has to face it!
                    dtheoryD = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                countsD = dtheoryD + self.bg #another artifact, incl this background effect somewhere
                unccountsD = np.sqrt(countsD)
                detactualD = rand.gauss(countsD,unccountsD)  #there is a lot of noise, present, updating it now. 
                if detactualD-self.bg < 0:
                    detactualD = self.bg
                    
                detcountsD = detactualD
                
                self.X_all.append([detcountsA,detcountsB,detcountsC,detcountsD])
                #coarse to fine optimization
                chiA = bf.quad_solver(detcountsA,self.normA,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA,self.bg)
                chiB = bf.quad_solver(detcountsB,self.normB,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA,self.bg)
                chiC = bf.quad_solver(detcountsC,self.normC,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA,self.bg)
                chiD = bf.quad_solver(detcountsD,self.normD,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA,self.bg)
                
                chisquared = np.add(np.add(chiA,chiB),np.add(chiC,chiD)) #adds it all up for total chi2
                
                #print("Chi squareds: " +str(chisquared))
                
                
                thetaloc, philoc, Aguess = bf.indexer(chisquared,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA)
                recvec = hp.ang2vec(np.deg2rad(thetaloc),np.deg2rad(philoc))
                locoffset = np.rad2deg(bf.angle(sourcexyz,recvec))
               # print("Loc offset = " + str(locoffset) + " deg")
                self.y_all.append([thetaloc,philoc,Aguess])
                locunc.append(locoffset)
                loop +=1
            #print("Avg loc offset = " + str(s.mean(locunc)) + " deg.")
            self.localizationerrors.append(s.mean(locunc))
        return self.localizationerrors

In [6]:
GRBtest = GRBgenerator.Sky(8,500)

In [4]:
import time

In [7]:
fastguy = FastCube(1000,45,alternating=False)

In [8]:
fastguy.response2GRB(GRBtest,test=False)



[7.3292824711931877,
 5.8827226629620828,
 5.6293480328272141,
 6.2047013742022843,
 7.046453454130714,
 6.2647255495657737,
 7.0302201836135092,
 6.5445746680877317,
 6.5323482190377042,
 5.8009995988892173,
 5.9611998610467056,
 7.4090003106551485,
 5.2450061867237494,
 6.1985524249569854,
 6.9216975712265274,
 7.6030550549546421,
 6.6609604727798972,
 6.2201329085550903,
 6.289614925969496,
 5.784731591704662,
 6.3575229503354533,
 5.5242497267562927,
 5.9459679416529427,
 6.7457134720081458,
 6.7298827320644961,
 6.2884254251088141,
 5.8775266433122066,
 5.4686316585384569,
 4.626687545486206,
 5.4178362289690822,
 5.1142028164178299,
 6.254777839192375,
 6.1787831885184614,
 5.7219569369339931,
 5.254308342242946,
 7.2507159642146277,
 5.9537105593176962,
 6.2168892606399675,
 5.9622105687985592,
 6.2809685717211412,
 5.7556045144293968,
 5.3120547394706401,
 6.0649719398617545,
 5.0671154004716561,
 5.1526584476582373,
 5.7013648565594011,
 5.7374703391049611,
 5.5599503766132807

In [9]:
len(fastguy.X_all)

23040

In [39]:
from sklearn.neural_network import MLPClassifier

In [40]:
model = MLPClassifier()

In [41]:
xall = np.array(fastguy.X_all)

yall = np.array(fastguy.y_all)

In [42]:
model.fit(xall,yall)

ValueError: Multioutput target data is not supported with label binarization

In [36]:
yes = model.predict(xall)

In [38]:
yes

array([[  73.95503307,  184.9512939 ,  657.66431359],
       [  71.09210407,  148.14759268,  650.8821978 ],
       [  77.96574515,  190.86817141,  677.90710947],
       ..., 
       [  53.40846173,  151.316692  ,  486.9772251 ],
       [  55.6340569 ,  141.247427  ,  500.00755958],
       [  53.94100807,  149.26853798,  487.83889734]])

Now the NN Part!

In [7]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow

Using TensorFlow backend.


In [10]:
xall = np.array(fastguy.X_all)

yall = np.array(fastguy.y_all)

In [60]:
x = np.array([1, 2, 3])

# row vector via reshape
np.shape(xall)

(1920, 4)

In [58]:
x = x.reshape((1, 3))
np.shape(x)

(1, 3)

192

In [ ]:
xtrain = np.split(xall)

In [9]:
model = Sequential()

In [47]:
model.add(Dense(12, input_dim=4, activation='linear'))
model.add(Dense(102, activation='linear'))
model.add(Dense(3, activation='relu'))

In [48]:
model.compile(loss='mse', optimizer='adam', metrics=['mae'])


Mean Absolute Error (MAE): MAE measures the average magnitude of the errors in a set of predictions, without considering their direction. It’s the average over the test sample of the absolute differences between prediction and actual observation where all individual differences have equal weight.



In [50]:
model.fit(X_train, y_train, epochs=50, batch_size=1000)


Epoch 1/50
1920/1920 [==============================] - 0s 10us/step - loss: 93055.9242 - mean_absolute_error: 217.0678
Epoch 2/50
1920/1920 [==============================] - 0s 9us/step - loss: 93078.3040 - mean_absolute_error: 217.1633
Epoch 3/50
1920/1920 [==============================] - 0s 10us/step - loss: 93090.3770 - mean_absolute_error: 217.2079
Epoch 4/50
1920/1920 [==============================] - 0s 12us/step - loss: 93088.5724 - mean_absolute_error: 217.2005
Epoch 5/50
1920/1920 [==============================] - 0s 13us/step - loss: 93076.6642 - mean_absolute_error: 217.1551
Epoch 6/50
1920/1920 [==============================] - 0s 11us/step - loss: 93056.0480 - mean_absolute_error: 217.0753
Epoch 7/50
1920/1920 [==============================] - 0s 11us/step - loss: 93041.2082 - mean_absolute_error: 217.0112
Epoch 8/50
1920/1920 [==============================] - 0s 10us/step - loss: 93030.2236 - mean_absolute_error: 216.9500
Epoch 9/50
1920/1920 [===================

In [16]:
len(X_train)

1920

In [51]:
model.predict(np.array([[1440.91398024,  1375.13145513,  1330.10174408,  1290.14606751]]))

array([[   0.        ,  172.46865845,    0.        ]], dtype=float32)

In [52]:
X_train[0]

array([ 1440.18999851,  1375.68773163,  1329.91204954,  1291.81638206])

In [53]:
X_train

array([[ 1440.18999851,  1375.68773163,  1329.91204954,  1291.81638206],
       [ 1401.01234871,  1407.4268673 ,  1349.20306728,  1303.81836212],
       [ 1415.39987524,  1449.78360876,  1369.12781504,  1313.04372026],
       ..., 
       [ 1016.53272338,  1003.58160115,  1000.        ,  1000.        ],
       [ 1038.4020694 ,  1008.50332082,  1000.        ,  1000.        ],
       [ 1019.51157439,  1040.71584411,  1000.        ,  1003.1049342 ]])

In [54]:
y_train

array([[  12.        ,   39.32773109,  454.54545455],
       [  12.        ,   63.52941176,  509.09090909],
       [  12.        ,   72.60504202,  509.09090909],
       ..., 
       [  66.        ,  245.04201681,  400.        ],
       [  78.        ,  260.16806723,  563.63636364],
       [  90.        ,  178.48739496,  836.36363636]])